## Midterm Assignment

In [1]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
from numerize import numerize as nz
from lets_plot import *
LetsPlot.setup_html()
import random

### Reading Data's from Waitrose file

In [2]:
all_file=[os.path.join('../DATA/waitrose',file) for file in os.listdir('../DATA/waitrose')
           if file.endswith('.csv')]
waitrose_df=pd.concat((pd.read_csv(file) for file in all_file))

In [3]:
waitrose_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 25418 entries, 0 to 1593
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data-product-id        25418 non-null  int64  
 1   data-product-name      25418 non-null  object 
 2   data-product-type      25418 non-null  object 
 3   data-product-on-offer  25418 non-null  bool   
 4   data-product-index     25408 non-null  float64
 5   image-url              25418 non-null  object 
 6   product-page           25418 non-null  object 
 7   product-name           25407 non-null  object 
 8   product-size           25363 non-null  object 
 9   item-price             25407 non-null  object 
 10  price-per-unit         24976 non-null  object 
 11  offer-description      7201 non-null   object 
 12  category               25418 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(10)
memory usage: 19.8 MB


In [4]:
waitrose_df.isnull().sum()

data-product-id              0
data-product-name            0
data-product-type            0
data-product-on-offer        0
data-product-index          10
image-url                    0
product-page                 0
product-name                11
product-size                55
item-price                  11
price-per-unit             442
offer-description        18217
category                     0
dtype: int64

### Dropping duplicates ,rename columns and drop unneccessary columns

In [5]:
waitrose_df=waitrose_df.drop_duplicates()
waitrose_df=waitrose_df.drop(columns=['data-product-index',
                                      'data-product-name',
                                      'data-product-type'])

In [6]:
waitrose_df=waitrose_df.rename(columns={
    'product-page':'page',
    'product-name':'name',
    'product-size':'size',
    'data-product-on-offer':'on-offer',
    'data-product-id':'id'
        })

### Redcuing id memory with changing dtype of id

In [7]:
waitrose_df['id'] = waitrose_df['id'].astype('int32')

In [8]:
def clean_item_price(item_price: str):    
    if ' each est.' in item_price:
        item_price=item_price.replace(' each est.','')
    
    if type(item_price) != str:
        pass
    elif '£' in item_price:
        item_price = item_price.replace('£', '')
    elif 'p' in item_price:
        item_price = item_price.replace('p', '')
        item_price = '0.' + item_price
    
    if '-' in item_price:
        item_price=item_price.split('-')[0]
    
        
    return float(item_price)

In [9]:
# apply the function to the item price column
waitrose_df['cleaned-price']=waitrose_df['item-price'].astype(str).apply(clean_item_price)


In [10]:
# apply the function to the item price column
waitrose_df[['cleaned-price','item-price']]

,cleaned-price,item-price
0,3.15,£3.15
1,3.15,£3.15
2,13.50,£13.50
3,0.95,95p
4,1.60,£1.60
...,...,...
1589,24.31,£24.31 each est.
1590,2.40,£2.40
1591,10.00,£10.00
1592,2.50,£2.50


In [11]:
select_rows=waitrose_df['item-price'].astype(str).str.contains('-')
waitrose_df[select_rows][['size','item-price']]

,size,item-price
291,(1kg-2kg),£55-£110 each est.
294,(1.4kg-1.8kg),£30-£38 each est.
299,(2kg-3.4kg),£54-£92 each est.
302,(6kg-7kg),£168-£196 each est.
305,(2.2kg-2.8kg),£31-£40 each est.
307,(1.8kg-2.2kg),£94-£115 each est.
311,(3.1kg-3.7kg),£35-£41 each est.
313,(0.9kg-1.1kg),£32-£39 each est.
318,(1.1kg-1.5kg),£24-£32 each est.
3237,(1.2kg-1.7kg),£42-£60 each est.


In [12]:
waitrose_df.loc[select_rows,'size']=waitrose_df.loc[select_rows,'size'].str.split('-').str[0].str.replace('(','')

In [13]:
waitrose_df['item-price']=waitrose_df['cleaned-price']
waitrose_df.drop(columns=['cleaned-price'],inplace=True)

In [14]:
waitrose_df.sort_values('id')

,id,on-offer,image-url,page,name,size,item-price,price-per-unit,offer-description,category
314,1023,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/taylors...,Taylors of Harrogate Yorkshire Gold 160 Bags,500g,7.10,£1.42/100g,NaN,"Tea, Coffee & Soft Drinks"
2403,1029,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/nestle-...,Nestlé Box Bowls Variety 6 Portion Pack,210g,2.30,£1.10/100g,NaN,Food Cupboard
513,1048,True,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/kellogg...,Kellogg's Corn Flakes Breakfast Cereal Large Pack,670g,2.75,41p/100g,save 45p. Was £3.20,Dietary & Lifestyle
152,1048,True,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/kellogg...,Kellogg's Corn Flakes Breakfast Cereal Large Pack,670g,2.75,41p/100g,save 45p. Was £3.20,Food Cupboard
364,1051,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/kellogg...,Kellogg's Crunchy Nut Breakfast Cereal Large Pack,840g,5.00,59.5p/100g,NaN,Food Cupboard
...,...,...,...,...,...,...,...,...,...,...
85,999167,True,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/merchan...,Merchant Gourmet Coconut & Lime Leaf Wholegrai...,250g,1.50,£6/kg,Introductory Offer.Will be £2.00,New
557,999432,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/andrex-...,Andrex Fresh & Sensitive Wipes,30Sheet,1.00,£3.34/100 sheets,NaN,Household
248,999432,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/andrex-...,Andrex Fresh & Sensitive Wipes,30Sheet,1.00,£3.34/100 sheets,NaN,New
247,999942,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/zebra-z...,Zebra Z-Grip Pack of 8 Assorted Ballpoint Pen,8Each,4.00,50p each,NaN,New


### TASK 01 – DATA CLEANING: 

In [ ]:
df_cleaned=waitrose_df.groupby('id')['category'].apply(list).reset_index()[['']]
# df_cleaned = (
#     waitrose_df.groupby("name", as_index=False)
#     .agg({
        
#         "category": lambda x: list(x),  # Collect all categories in a list

#     })
# )
df_cleaned

,id,category
0,1023,"[Tea, Coffee & Soft Drinks]"
1,1029,[Food Cupboard]
2,1048,"[Dietary & Lifestyle, Food Cupboard]"
3,1051,"[Dietary & Lifestyle, Food Cupboard]"
4,1052,"[Dietary & Lifestyle, Food Cupboard]"
...,...,...
16050,998674,"[New, Tea, Coffee & Soft Drinks]"
16051,999031,"[Fresh & Chilled, New]"
16052,999167,"[Food Cupboard, New]"
16053,999432,"[Household, New]"


In [26]:
# Merge the dataframes on 'id' (or another common key)
df_cleaned = df_cleaned.merge(waitrose_df[['id', 'name', 'size', 'item-price', 'price-per-unit']], on='name', how='right')

In [27]:
df_cleaned

,name,category,id,size,item-price,price-per-unit
0,Organix Raspberry & Apple Soft Oaty Bars,"[Baby, Child & Parent]",525635,6x23g,3.15,£2.29/100g
1,Organix Carrot Cake Oaty Bars,"[Baby, Child & Parent]",557746,6x23g,3.15,£2.29/100g
2,Aptamil 2 Follow On Milk,"[Baby, Child & Parent, Baby, Child & Parent, B...",32062,800g,13.50,£16.88/kg
3,Essential Baby Wipes,"[Baby, Child & Parent, Summer, Toiletries, Hea...",767801,80s,0.95,1.2p each
4,Organix Apple Rice Cakes,"[Baby, Child & Parent, Baby, Child & Parent, D...",514054,40g,1.60,£4/100g
...,...,...,...,...,...,...
25373,Duchy Organic British Lamb Half Leg,"[Best of British, Fresh & Chilled, Organic Sho...",18427,Typical weight 0.94kg,24.31,£26.00/kg
25374,Cooks' Ingredients Wok Oil,"[Food Cupboard, Waitrose Brands]",11606,250ml,2.40,96p/100ml
25375,No. 1 Badoz Vacherin Du Haut-Doubs AOP French ...,"[Fresh & Chilled, Waitrose Brands]",6903,350g,10.00,£28.58/kg
25376,Cooks' Ingredients White Marzipan,"[Food Cupboard, Waitrose Brands]",6125,500g,2.50,£5/kg


In [33]:
df_waitorse=pd.merge(waitrose_df,df_cleaned,on='id',how='right')

In [34]:
df_waitorse

,id,on-offer,image-url,page,name_x,size_x,item-price_x,price-per-unit_x,offer-description,category_x,name_y,category_y,size_y,item-price_y,price-per-unit_y
0,525635,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Raspberry & Apple Soft Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent",Organix Raspberry & Apple Soft Oaty Bars,"[Baby, Child & Parent]",6x23g,3.15,£2.29/100g
1,557746,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Carrot Cake Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent",Organix Carrot Cake Oaty Bars,"[Baby, Child & Parent]",6x23g,3.15,£2.29/100g
2,32062,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/aptamil...,Aptamil 2 Follow On Milk,800g,13.50,£16.88/kg,NaN,"Baby, Child & Parent",Aptamil 2 Follow On Milk,"[Baby, Child & Parent, Baby, Child & Parent, B...",800g,13.50,£16.88/kg
3,767801,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Baby Wipes,80s,0.95,1.2p each,NaN,"Baby, Child & Parent",Essential Baby Wipes,"[Baby, Child & Parent, Summer, Toiletries, Hea...",80s,0.95,1.2p each
4,767801,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Baby Wipes,80s,0.95,1.2p each,NaN,Summer,Essential Baby Wipes,"[Baby, Child & Parent, Summer, Toiletries, Hea...",80s,0.95,1.2p each
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49933,6125,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/cooks-i...,Cooks' Ingredients White Marzipan,500g,2.50,£5/kg,NaN,Waitrose Brands,Cooks' Ingredients White Marzipan,"[Food Cupboard, Waitrose Brands]",500g,2.50,£5/kg
49934,707668,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/duchy-o...,Duchy Organic Pinot Grigio Bag in Box,2.25L,21.49,£7.17/75cl,NaN,"Beer, Wine & Spirits",Duchy Organic Pinot Grigio Bag in Box,"[Beer, Wine & Spirits, Organic Shop, Summer, W...",2.25L,21.49,£7.17/75cl
49935,707668,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/duchy-o...,Duchy Organic Pinot Grigio Bag in Box,2.25L,21.49,£7.17/75cl,NaN,Organic Shop,Duchy Organic Pinot Grigio Bag in Box,"[Beer, Wine & Spirits, Organic Shop, Summer, W...",2.25L,21.49,£7.17/75cl
49936,707668,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/duchy-o...,Duchy Organic Pinot Grigio Bag in Box,2.25L,21.49,£7.17/75cl,NaN,Summer,Duchy Organic Pinot Grigio Bag in Box,"[Beer, Wine & Spirits, Organic Shop, Summer, W...",2.25L,21.49,£7.17/75cl


In [18]:
df_waitorse.rename(columns={
    'category_x':'category',
    'category_y':"categories",
    'item-price_x':'item-price'
},inplace=True)

In [19]:
df_waitorse

,id,on-offer,image-url,page,name,size,item-price,price-per-unit,offer-description,category,categories
0,525635,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Raspberry & Apple Soft Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent","[Baby, Child & Parent]"
1,557746,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Carrot Cake Oaty Bars,6x23g,3.15,£2.29/100g,NaN,"Baby, Child & Parent","[Baby, Child & Parent]"
2,32062,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/aptamil...,Aptamil 2 Follow On Milk,800g,13.50,£16.88/kg,NaN,"Baby, Child & Parent","[Baby, Child & Parent]"
3,767801,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Baby Wipes,80s,0.95,1.2p each,NaN,"Baby, Child & Parent","[Baby, Child & Parent, Summer, Toiletries, Hea..."
4,514054,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Apple Rice Cakes,40g,1.60,£4/100g,NaN,"Baby, Child & Parent","[Baby, Child & Parent, Dietary & Lifestyle]"
...,...,...,...,...,...,...,...,...,...,...,...
25373,18427,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/duchy-o...,Duchy Organic British Lamb Half Leg,Typical weight 0.94kg,24.31,£26.00/kg,NaN,Waitrose Brands,"[Best of British, Fresh & Chilled, Organic Sho..."
25374,11606,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/cooks-i...,Cooks' Ingredients Wok Oil,250ml,2.40,96p/100ml,NaN,Waitrose Brands,"[Food Cupboard, Waitrose Brands]"
25375,6903,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/no-1-ba...,No. 1 Badoz Vacherin Du Haut-Doubs AOP French ...,350g,10.00,£28.58/kg,NaN,Waitrose Brands,"[Fresh & Chilled, Waitrose Brands]"
25376,6125,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/cooks-i...,Cooks' Ingredients White Marzipan,500g,2.50,£5/kg,NaN,Waitrose Brands,"[Food Cupboard, Waitrose Brands]"


### TASK 02 – COMPARATIVE ANALYSIS

In [20]:
duplicated_pro=(
   df_waitorse.groupby('name')['category']
    .nunique()
    .reset_index()
    .query('category > 5')  
)
duplicated_pro

,name,category
3419,Duchy Organic British FR Eggs Very Large,6
3420,Duchy Organic British Free Range Eggs,6
3421,Duchy Organic British Free Range Eggs Large,6
3422,Duchy Organic British Free Range Medium Eggs,6
3510,Duchy Organic Homogenised Semi-Skimmed Milk 4 ...,6
3512,Duchy Organic Houmous,6
3608,Duchy Organic Unhomogenised Whole Milk 4 Pints,6
4234,Essential Cucumber,6
4404,Essential Limes,6
4678,Essential Sweetcorn,6


In [21]:
essential_cucumber=df_waitorse.loc[df_waitorse['name'].str.contains("Cucumber",na=False)]

In [22]:
remove_cum=["Margarita","Tonic","Smoothie","Juice","Water","Soup","Blend","Wipes","Dove","Pressé","Roll","Spears","Haimisha"]

In [23]:
essential_cucumber=essential_cucumber.loc[~essential_cucumber['name'].str.contains('|'.join(remove_cum), case=False)]

In [24]:
essential_cucumber

,id,on-offer,image-url,page,name,size,item-price,price-per-unit,offer-description,category,categories
2510,86468,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Cucumber,Each,0.95,95p each,NaN,"Beer, Wine & Spirits","[Beer, Wine & Spirits, Dietary & Lifestyle, Ev..."
2916,86468,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Cucumber,Each,0.95,95p each,NaN,Dietary & Lifestyle,"[Beer, Wine & Spirits, Dietary & Lifestyle, Ev..."
2958,476506,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/waitros...,Waitrose Midi Cucumber,each,0.65,65p each,NaN,Dietary & Lifestyle,"[Dietary & Lifestyle, Summer]"
2982,564598,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/duchy-o...,Duchy Organic Salad Cucumber,each,1.40,£1.40 each,NaN,Dietary & Lifestyle,"[Dietary & Lifestyle, Fresh & Chilled, Organic..."
3010,86469,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Large Cucumber,Each,1.10,£1.10 each,NaN,Dietary & Lifestyle,"[Dietary & Lifestyle, Everyday Value, Fresh & ..."
3281,85315,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/duchy-o...,Duchy Organic Ridge Cucumber,Each,1.65,£1.65 each,NaN,Dietary & Lifestyle,"[Dietary & Lifestyle, Fresh & Chilled, Organic..."
6250,86468,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Cucumber,Each,0.95,95p each,NaN,Everyday Value,"[Beer, Wine & Spirits, Dietary & Lifestyle, Ev..."
6271,86469,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Large Cucumber,Each,1.10,£1.10 each,NaN,Everyday Value,"[Dietary & Lifestyle, Everyday Value, Fresh & ..."
7079,634261,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/mrs-els...,Mrs Elswood Sweet Cucumber Slices,drained 310g,2.80,90.3p/100g,NaN,Food Cupboard,[Food Cupboard]
10582,86468,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Cucumber,Each,0.95,95p each,NaN,Fresh & Chilled,"[Beer, Wine & Spirits, Dietary & Lifestyle, Ev..."


In [25]:
# Step 1: Filter Everyday Value products
everyday_value =df_waitorse[df_waitorse["category"].str.contains("Everyday Value", na=False, case=False)].head(10)
branded = df_waitorse[~df_waitorse["category"].str.contains("Everyday Value", na=False, case=False)].head(10)

In [26]:
dup=pd.concat([everyday_value,branded],axis=0,ignore_index=True)

In [27]:
(
    ggplot(essential_cucumber, aes(x='category', y='item-price', fill='name')) +
    geom_bar(stat = "identity") + 
    scale_y_continuous(limits=(0, 15)) +
    labs(
        title="Item Price Comparison: Everyday Value vs Branded",
        x="Product Name",
        y="Item Price",
        fill="Category"
    ) +theme(axis_text_x=element_text(size=13,face="bold"),
          axis_text_y=element_text(size=13,face="bold"),
          plot_title=element_text(size=30,face="bold"),
          axis_title_x=element_text(size=22,face="bold"),
          axis_title_y=element_text(size=22,face="bold")) + 
    ggsize(1000, 800) 
)


In [28]:
(ggplot(essential_cucumber, aes(x='category', y='item-price', color='name',size='item-price')) +
    geom_point() +  
     scale_size(range=(6, 15))+
    labs(
        title="Price Distribution: Everyday Value vs Branded",
        x="Category",
        y="Item Price",
        color="Name",
        size="Item-Price"
    )+    theme(axis_text_x=element_text(size=15,face="bold"),
          axis_text_y=element_text(size=15,face="bold"),
          plot_title=element_text(size=30,face="bold"),
          axis_title_x=element_text(size=22,face="bold"),
          axis_title_y=element_text(size=22,face="bold"))
    +ggsize(1000,800)
)

In [29]:
(
    ggplot(essential_cucumber, aes(x='name', y='item-price', color='category')) +
    geom_line() +  # Box plot for distribution comparison
    scale_y_continuous(limits=(0, 15)) +  # Adjust y-axis to fit data
    labs(
        title="Price Distribution: Everyday Value vs Branded",
        x="Category",
        y="Item Price",
        fill="Category"
    ) +
    theme(axis_text_x=element_text(angle=0, hjust=0.5)) +  # Straighten x-axis labels for clarity
    ggsize(800, 600)  # Standardize plot size
)


In [30]:
plot_df=dup[['id','name','size','item-price','category']]
plot_df=plot_df.sort_values('item-price')


In [31]:
(
    ggplot(plot_df, aes(x='name', y='item-price', fill='category')) +
    geom_bar(stat='identity')+  # Column plot for categorical comparison
    scale_y_continuous(limits=(0, 15)) +  # Adjust y-axis to fit data
    labs(
        title="Item Price Comparison: Everyday Value vs Branded",
        x="Product Name",
        y="Item Price",
        fill="Category"
    ) +
    theme(axis_text_x=element_text(angle=45, hjust=1)) +  # Rotate x-axis labels for readability
    ggsize(900, 600)  # Increase width for better display of names
)

In [32]:
(
    ggplot(plot_df, aes(x='name', y='item-price', color='category', group='category')) +
    geom_line(size=1.5) +  
    geom_point(size=4) +  
    # scale_y_continuous(name='item-price',
    #     breaks=[0, 0.5, 1, 2, 3,4,5,6,7,8,9,10,11, 12,13]  # Custom breaks
    #     # labels=['0', '0.5', '1', '2', '3', '4'13+'] 
    #     ) +  # Adjust y-axis to fit data
    labs(
        title="Item Price Comparison: Everyday Value vs Branded (Line Chart)",
        x="Product Name",
        y="Item Price",
        color="Category"
    ) +
    theme(axis_text_x=element_text(angle=45, hjust=1)) +  
    ggsize(900, 600)  
)
